In [10]:
import mbuild as mb
from foyer import Forcefield
from mbuild.lib.recipes.polymer import Polymer
import unyt as u

import mosdef_cassandra as mc

## Prepare recipes

### Exercise 1

In [11]:
class _CH2(mb.Compound):
    def __init__(self):
        super().__init__()
        self.name = "_CH2"
        # Create a partilce name _CH2
        _ch2 = mb.Compound(name="_CH2")
        self.add(_ch2, "_CH2")
        
        # Create ports to be attached to the _CH2 particles
        port_up = mb.Port(anchor=_ch2, orientation=[0, 0, 1], separation=0.07)
        port_down = mb.Port(anchor=_ch2, orientation=[0, 0, -1], separation=0.07)
        self.add(port_up, "up")
        self.add(port_down, "down")
        
class _CH3(mb.Compound):
    def __init__(self):
        super().__init__()
        self.name = "_CH3"
        
        # Create a particle name _CH3
        _ch3 = mb.Compound(name="_CH3")
        self.add(_ch3, "_CH3")
        
        # Create a port to be attached to the _CH3 particle
        port_up = mb.Port(anchor=_ch3, orientation=[0, 0, 1], separation=0.07)
        self.add(port_up, "up")

            

### Exercise 2 

In [12]:
class CG_Alkane(mb.Compound):
    def __init__(self, n):
        super().__init__()
        self.name = "CG_Alkane"
    
        if n == 1: 
            _ch4 = mb.Compound(name="_CH4")
            self.add(_ch4, "_CH4")
        elif n == 2:
            _ch3a = _CH3() 
            _ch3b = _CH3() 
            self.add(_ch3a, "_CH3_a")
            self.add(_ch3b, "_CH3_b")
            mb.force_overlap(move_this=_ch3a, 
                             from_positions=_ch3a["up"], 
                             to_positions=_ch3b["up"])
        else:
            _ch3top = _CH3()
            self.add(_ch3top, "link0")
            for i in range(n - 2):
                _ch2 = _CH2()
                mb.force_overlap(move_this=_ch2, 
                                 from_positions=_ch2["down"],
                                 to_positions=self[f"link{i}"]["up"])
                self.add(_ch2, f"link{i+1}")
                
            _ch3bot = _CH3()
            self.add(_ch3bot, f"link{i+2}")
            mb.force_overlap(move_this=_ch3bot, 
                             from_positions=_ch3bot["up"], 
                             to_positions=self[f"link{i+1}"]["up"])


In [13]:
from mbuild.lib.recipes import Polymer

cg_alkane = Polymer(monomers=[_CH2()], end_groups=[_CH3(), _CH3()])
cg_alkane.build(n=5)

### Exercise 3

In [14]:
n_pentane = CG_Alkane(5)
pentane_box = mb.fill_box(compound=n_pentane, 
                          n_compounds=150, 
                          box=[3.2, 3.2, 3.2],
                          seed=12345, 
                          edge=0.2)

/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/conversion.py:1747: UserWarning: No element attribute associated with '<_CH3 pos=([0. 0. 0.]), 0 bonds, id: 5981734800>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/conversion.py:1747: UserWarning: No element attribute associated with '<_CH2 pos=([ 9.6244e-18 -8.3696e-18  1.4000e-01]), 0 bonds, id: 5974406576>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)


### Exercise 4

In [15]:
import foyer 
from foyer import Forcefield

trappe = Forcefield(name="trappe-ua")
pentane_box.energy_minimize(forcefield="trappe-ua", 
                            steps=10**6)

pentane_box.visualize()


/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/conversion.py:1747: UserWarning: No element attribute associated with '<_CH3 pos=([0.8896 2.0615 2.5952]), 0 bonds, id: 5977792624>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/conversion.py:1747: UserWarning: No element attribute associated with '<_CH2 pos=([0.9855 1.969  2.6098]), 0 bonds, id: 5977794448>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/conversion.py:1747: UserWarning: No element attribute associated with '<_CH3 pos=([1.0477 2.1153 2.794 ]), 0 bonds, id: 5988696704>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)
/Users/quachcd/mambaforge/envs/general/lib/python

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Exercise 5a - Simulation with MoSDeF Cassandra

In [9]:
# Load forcefields
trappe = Forcefield(name="trappe-ua")

# Use foyer to apply forcefields
typed_pentane = trappe.apply(n_pentane)
species_list = [typed_pentane]

# Create box and species list
box_list = [pentane_box]
mols_in_boxes = [[150]]

system = mc.System(box_list, species_list, mols_in_boxes=mols_in_boxes)
moveset = mc.MoveSet("npt", species_list)

moveset.prob_volume = 0.010

thermo_props = [
    "energy_total",
    "energy_intervdw",
    "pressure",
    "volume",
    "nmols",
    "mass_density",
]

custom_args = {
    "run_name": "equil",
    "charge_style": "none",
    "rcut_min": 2.0 * u.angstrom,
    "vdw_cutoff": 14.0 * u.angstrom,
    "units": "sweeps",
    "steps_per_sweep": 450,
    "coord_freq": 50,
    "prop_freq": 10,
    "properties": thermo_props,
}


mc.run(
    system=system,
    moveset=moveset,
    run_type="equilibration",
    run_length=2400,
    pressure=14.02 * u.bar,
    temperature=372 * u.K,
    **custom_args,
)

Using the following executables for Cassandra:
Python: /Users/quachcd/mambaforge/envs/general/bin/python
library_setup: /Users/quachcd/mambaforge/envs/general/bin/library_setup.py
Cassandra: /Users/quachcd/mambaforge/envs/general/bin/cassandra.exe


/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/conversion.py:1747: UserWarning: No element attribute associated with '<_CH3 pos=([1.0931 1.9997 2.7119]), 0 bonds, id: 5985513920>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/conversion.py:1747: UserWarning: No element attribute associated with '<_CH2 pos=([1.1224 1.9023 2.8275]), 0 bonds, id: 5985514640>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)


Generating fragment libraries...
Running Cassandra...


CassandraRuntimeError: Cassandra exited with an error, see mosdef_cassandra_2022-06-29_14:56:20.001375.log for details.

### Exercise 5b - Simulation with GROMACS

In [29]:
pentane_box.save("gmx_sim/init.gro", overwrite=True)
pentane_box.save("gmx_sim/init.top", forcefield_name="trappe-ua")

/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/conversion.py:1747: UserWarning: No element attribute associated with '<_CH3 pos=([1.0477 2.1153 2.794 ]), 0 bonds, id: 5977792624>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/mbuild/conversion.py:1747: UserWarning: No element attribute associated with '<_CH2 pos=([1.0741 1.9938 2.8849]), 0 bonds, id: 5977794448>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _CH4
  warnings.warn(
/Users/quachcd/mambaforge/envs/general/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _CH3

In [39]:
nvt_mdp = """
integrator          = md
nsteps              = 100000
dt                  = 0.001

comm-mode           = Linear

nstxout             = 10000
nstvout             = 10000
nstenergy           = 1000
nstlog              = 1000

cutoff-scheme       = Verlet
ns_type             = grid
nstlist             = 10
pbc                 = xyz 

vdwtype         = Cut-off
vdw-modifier    = None
rvdw            = 1.4 

coulombtype             = Cut-off
coulomb-modifier        = None
rcoulomb                = 1.4 

tcoupl              = nose-hoover
tc-grps             = System
tau_t               = 1 
ref_t               = 372.0

pcoupl              = no

DispCorr            = EnerPres
"""
with open("gmx_sim/nvt.mdp", "w") as f:
    f.write(nvt_mdp)

In [40]:
%cd gmx_sim
!gmx grompp -f nvt.mdp -o nvt.tpr -c init.gro -p init.top --maxwarn 1
!gmx mdrun -v -deffnm nvt -s nvt.tpr -cpi nvt.cpt
%cd ..

/Users/quachcd/Documents/tutorials/fomms_workshop/FOMMS-workshop/example_workflow/gmx_sim
                      :-) GROMACS - gmx grompp, 2020.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Ma

### Exercise 6

In [41]:
import numpy as np
import pylab as plt

data = np.genfromtxt("equil.out.prp", skip_header=3)

plt.rcParams['font.family'] = "DIN Alternate"
font = {'family' : 'DIN Alternate',
        'weight' : 'normal',
        'size'   : 12}

fig, ax = plt.subplots(1, 1)

ax.spines["bottom"].set_linewidth(3)
ax.spines["left"].set_linewidth(3)
ax.spines["right"].set_linewidth(3)
ax.spines["top"].set_linewidth(3)

ax.title.set_text('Control plot')
ax.set_xlabel(r'MC sweeps')
ax.set_ylabel('Density $(kg / m{^3})$')
ax.yaxis.tick_left()
ax.yaxis.set_label_position('left')
ax.axhline(y=541, color='r', linestyle='-', label='~TraPPE-UA Density')

ax.plot(data[:, 0], data[:,-1], "-", color='lightgray', label='Density')
ax.legend(loc="best")
plt.show()

OSError: equil.out.prp not found.